In [ ]:
import pandas as pd

df_patients = pd.read_csv("PATIENTS.csv")
df_admissions = pd.read_csv("ADMISSIONS.csv")

# Convert date columns to datetime
df_patients['DOB'] = pd.to_datetime(df_patients['DOB'], errors='coerce', infer_datetime_format=True)
df_admissions['ADMITTIME'] = pd.to_datetime(df_admissions['ADMITTIME'], errors='coerce', infer_datetime_format=True)
df_admissions['DISCHTIME'] = pd.to_datetime(df_admissions['DISCHTIME'], errors='coerce', infer_datetime_format=True)

# Calculate the difference in days between DISCHTIME and ADMITTIME
df_admissions['StayDays'] = (df_admissions['DISCHTIME'] - df_admissions['ADMITTIME']).dt.days

# Sum the StayDays for each SUBJECT_ID
total_admitted_days = df_admissions.groupby('SUBJECT_ID')['StayDays'].sum().reset_index()
total_admitted_days.columns = ['SUBJECT_ID', 'TotalAdmittedDays']

# Create the new column 'ReAdmitCount' which counts the number of times each SUBJECT_ID appears
df_admissions['ReAdmitCount'] = df_admissions.groupby('SUBJECT_ID')['SUBJECT_ID'].transform('count')

# Merge total admitted days and ReAdmitCount back into the main dataframe
df_admissions = pd.merge(df_admissions, total_admitted_days, on='SUBJECT_ID', how='left')

# Sort df_admissions by SUBJECT_ID and ADMITTIME
df_admissions_sorted = df_admissions.sort_values(by=['SUBJECT_ID', 'ADMITTIME'])

# Merge df_patients and sorted df_admissions on SUBJECT_ID
merged_df = pd.merge(df_admissions_sorted, df_patients[['SUBJECT_ID', 'DOB']], on='SUBJECT_ID', how='left')

# Determine the first admission time for each patient
merged_df['FirstAdmitTime'] = merged_df.groupby('SUBJECT_ID')['ADMITTIME'].transform('min')

# Extract the year from the first ADMITTIME and DOB
merged_df['YearOfFirstAdmit'] = merged_df['FirstAdmitTime'].dt.year
merged_df['YearOfBirth'] = merged_df['DOB'].dt.year

# Calculate the age by subtracting the year of DOB from the year of first ADMITTIME
merged_df['AGE'] = merged_df['YearOfFirstAdmit'] - merged_df['YearOfBirth']

# Select the relevant columns and remove duplicates
final_df = merged_df[['SUBJECT_ID','AGE', 'TotalAdmittedDays', 'ReAdmitCount']].drop_duplicates(subset='SUBJECT_ID')
final_df = final_df.merge(df_admissions[['SUBJECT_ID', 'ETHNICITY']], on='SUBJECT_ID', how='left')

# Drop records with NaN values in the 'age' column
final_df = final_df.dropna(subset=['AGE'])

final_df['AVG_ADMITDAYS'] = final_df['TotalAdmittedDays'] / final_df['ReAdmitCount']

final_df = final_df.drop_duplicates(subset='SUBJECT_ID', keep='first')

final_df.to_csv('FinalData.csv')